<a href="https://colab.research.google.com/github/eriksonferreira/ia-search-algorithms/blob/main/atp3_eferreira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

# Avaliação teórico-prática 3 (ATP3)

*Para questões discursivas, você pode escrever em formato Markdown/LaTeX ou, preferencialmente, incluir uma fotografia da solução do exercício.*

___
**1.** **(2 pontos)** Pesquise sobre e forneça a motivação para o de números complexos na formulação da Transformada de Fourier.
___


```

A motivação para usar números complexos na Transformada de Fourier é principalmente a simplificação matemática e a riqueza de informações que eles oferecem. Ao representar senos e cossenos, componentes fundamentais da série de Fourier, como exponenciais complexas (conforme a identidade de Euler), a matemática envolvida se torna mais tratável. Além disso, os números complexos permitem capturar tanto a amplitude quanto a fase das frequências de uma função, o que é crucial para uma análise detalhada e completa de sinais em várias aplicações práticas.

...

___
**2.** **(3 pontos)** Explique, com suas palavras, o Teorema da Convolução e sua importância no contexto de processamento de sinais.
___

O Teorema da Convolução é um conceito central no processamento de sinais que relaciona a convolução de duas funções no domínio do tempo com a multiplicação de suas transformadas de Fourier no domínio da frequência. Essa relação é crucial porque permite realizar operações complexas de convolução de maneira mais simples e eficiente no domínio da frequência. A aplicação prática desse teorema é amplamente vista em áreas como filtragem de sinais, processamento de áudio e imagem, reduzindo significativamente a complexidade computacional e aumentando a eficiência desses processos.

...

___
**3.** **(10 pontos)** O objetivo desta atividade é implementar um filtro Notch para remover ruído periódico de imagens. O processo será dividido em quatro passos:

1. Analisar o espectro de Fourier $F$ da imagem.
2. Identificar as localizações dos picos em $F$.
3. Construir um filtro notch de rejeição $H$ no domínio de Fourier, cujos centros estão nos picos.
4. Utilizar $H$ para filtrar $F$ e obter o resultado.

Para cada centro $(u,v)$, utilize filtros de Butterworh ($H_b$), cuja definição é

$$H_b(u,v) = \frac{1}{1+[D_0/D(u,v)]^{2n}},$$

onde $n$ é a ordem do filtro e $D_0$ o parâmetro que controla a abertura do filtro (*cut-off distance*).

Para cumprir esta tarefa, você deve utilizar uma imagem de teste corrompida com ruído periódico em diversas frequências. Além disso, você deve implementar a função `apply_notch_filter`.

___

In [ ]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, io
from scipy.fft import fft2, fftshift, ifft2, ifftshift
from skimage.color import rgb2gray

def spectrum(image):
    f_uv = fftshift(fft2(image))
    spec = np.log(np.abs(f_uv) + 1)
    return spec, f_uv


def apply_notch_filter(image, centers, f_uv):
    # Create an empty 'output' image with the same dimensions as the 'image' input.
    output = image.copy()
    D0 = 100
    n = 2
    h = image.shape[0]
    w = image.shape[1]


    H = np.ones((h, w), dtype=np.float32)

    for u, v in centers:
        for i in range(h):
            for j in range(w):
                D = np.sqrt((i - u) ** 2 + (j - v) ** 2) + 1e-10  # Adiciona um pequeno valor para evitar divisão por zero
                H[i, j] *= 1 / (1 + (D0 / D) ** (2 * n))

    f_uv_filtered = f_uv * H
    output = np.abs(ifft2(ifftshift(f_uv_filtered)))
    output = np.clip(output, 0, 255)  # Normaliza a imagem
    return output.astype(image.dtype)


def onclick(event):
    # Check whether one clicked inside the spectrum plot
    if event.inaxes in [axs[1]]:
        x, y = event.xdata, event.ydata
        if x is not None and y is not None:
            # Update notch_centers
            h = test_image.shape[0]
            w = test_image.shape[1]
            notch_centers.append((int(x), int(y)))
            notch_centers.append((w-x, h-y))

            # Perform filtering based on the clicked position and the 'image'.
            filtered = apply_notch_filter(test_image, notch_centers, f_uv)

            # Update the displayed image with the filtered result.
            img_plot.set_data(filtered)

            # Show notch centers
            axs[1].scatter(*zip(*notch_centers), marker="o", color="red")

            plt.draw()


notch_centers = []

# Load a test image: REPLACE THIS IMAGE WITH ONE CORRUPTED BY PERIODIC NOISE
test_image_rgb = io.imread('ruido_periodico.png', plugin='matplotlib')
test_image = rgb2gray(test_image_rgb[..., :3])

# Create a figure with two subplots for the image and its spectrum
spec, f_uv = spectrum(test_image)

fig, axs = plt.subplots(1, 2, figsize=(10, 20))
img_plot = axs[0].imshow(test_image, cmap="gray")
spec_plot = axs[1].imshow(spec, cmap="gray")


# Connect the click event to the 'onclick' function, passing the image and plot.
cid = fig.canvas.mpl_connect("button_press_event", onclick)